<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_Padel_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu May 12 20:10:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


import os 
# path = '/content/gdrive/MyDrive/Colab Notebooks/TrackNetv2/3_in_3_out'
#path = '/content/gdrive/MyDrive/Colab Notebooks/TrackNet/Code_Python3/TrackNet_Three_Frames_Input/'

!pwd

Mounted at /content/gdrive
/content


In [3]:
os.chdir('/content/')
!pwd

/content


In [39]:
#!git clone https://github.com/VictorUceda/TrackNet-Padel.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git pull

Already up to date.


In [ ]:
!pip3 install filterpy sktime piexif imutils Pillow focal_loss

### Test accuracy

In [ ]:
file_name = 'Final_PremierQatar_Tie'
point_n = 0
for point_n in range(13):
  video = 'raw_data/points/'+file_name+"_"+str(point_n)+".mp4"
  print(video)
  csv = 'raw_data/points/'+file_name+"_"+str(point_n)+".csv"
  res = 'raw_data/points/'+file_name+"_"+str(point_n)+".res.txt"
  !python ../TrackNet-Padel/predict.py --video_path={video} --label_path={csv} 2>&1 | tee {res}

2022-05-13 07:52:46.463322: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......
Number of true positive: 126
Number of true negative: 83
Number of false positive FP1: 31
Number of false positive FP2: 71
Number of false negative: 17
Accuracy: 0.6371951219512195
Precision: 0.5526315789473685
Recall: 0.8811188811188811
Total Time: 35.0849187374115
(ACC + Pre + Rec)/3: 0.6903151940058231
Done......
2022-05-13 07:53:32.731800: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......
Number of true positive: 233
Number of true negative: 51
Number of false positive FP1: